In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sdes.sdes import SDE, OrnsteinUhlenbeck, BrownianMotion
from sdes.auxiliary_bridges import DelyonHuBridge

In [2]:
def plot_simulations(X, t_start = 0., end_point=True):
    fig, ax = plt.subplots()
    names = X.dtype.names if end_point else X.dtype.names[:-1]
    t_s = [float(name) + t_start for name in names]
    data = np.stack([X[name] for name in names])
    ax.plot(t_s, data)
    return fig, ax

def struct_array_to_array(A):
    return np.array([A[name][0] for name in A.dtype.names])

def check_struct_array_equal(X_1, X_2):
    names = X_1.dtype.names
    check_array_equal = lambda x, y: np.all(np.isclose(x, y))
    check_list = [check_array_equal(X_1[name], X_2[name]) for name in names]
    return np.all(check_list)

In this notebook, we apply various tests to the auxiliary bridge transforms $F_{t', t''}^{(DH)}(x_{t'}, x_{t''},\quad  \cdot  \quad )$ and its inverse that are used in the implementation of the SMC-based algorithms that are designed on pathspace.

Assuming that the particles are stored in $Z, X$ form:

The map $F$ is needed to:

- Generate simulations from the kernel $P_i[dx_i, x_{i-1}]$ when using a bootstrap formalism.

The map $F^{-1}$ is used for 

- Translate the final particles into simulations from SDEs once the inference algorithm has been run.
- Evaluating the (log) density of the bridge (required for smoothing algorithms when using bootstrap and in the particle filter when using guided)

We will use as a particular running example the Ornstein-Uhlenbeck SDE from $t'=1$ to $t''=2$. We will pick reasonable values of $x_{t'}$ and $x_{t''}$ based on simulation. 

- Setup an auxiliary SDE. Generate simulations from the auxiliary SDE.
    - Amend the code of the auxiliary SDE simulators so that it is possible for the simulations to end at the correct point.
- Demonstrate the utility of the $F$ map by applying it to the simulated functions.


# Test 1 - Auxiliary Bridge transform for single $x_s$, $x_t$

We work with the univariate Ornstein-Uhlenbeck process as as the running example, which is defined by the following SDE:

$$dX_t = \rho(\mu - X_t)dt + \phi dW_t$$

Thus, the drift coefficient of this SDE is $b(t, x) = \rho(\mu - x)$ and the the diffusion coefficient is given by $\sigma(t, x) = \phi$.

Using Stochastic Calculus, this SDE can be solved analytically, and has solution:

$$X_t = X_0 e^{-\rho t} + \mu(1 - e^{-\rho t}) + \phi \int_0^t e^{-\rho (t-s)}dW_s$$

Thus, the mean function of the process are given by:

$$E[X_t|X_0] = X_0 e^{-\rho t} + \mu(1 - e^{-\rho t})$$

The parameter (vector) of the SDE is given by $\theta = (\rho,\phi, \mu)$. We let $\theta = (1, 0, 0.2)$ and we let $X_0=2$:

In [3]:
rho = 1.; mu = 0.; phi = 0.2; x_0 = 2.

ou_sde_params = {'rho': rho,
                 'mu': mu,
                 'phi': phi
                  }

ou_sde = OrnsteinUhlenbeck(**ou_sde_params)

The stochastic differential equation given above, for any fixed values of $t'$, $t''$, $x_{t'}$ and $x_{t''}$ implies a particular conditional distribution of a bridge $\mathbb{P}_{x_{t'}, x_{t''}}^{t', t''}$. This conditional distribution of a bridge is equivalent to the distribution of the solution of the (Delyon and Hu) auxiliary bridge SDE given by:

$$d\tilde{X}_t = \bigg[b(t+t', \tilde{X}_t) + \frac{x_{t''} - \tilde{X}_t}{\Delta t-t}\bigg]dt + \sigma(t+t', \tilde{X}_t)dW_t$$

With starting point $\tilde{X}_0 = x_{t'}$ and end point in time $\Delta t = t'' - t'$. This distribution is denoted $\mathbb{Q}_{x_{t'}, x_{t''}}^{t', t''}$. Both distributions $\mathbb{P}_{x_{t'}, x_{t''}}^{t', t''}$ and $\mathbb{Q}_{x_{t'}, x_{t''}}^{t', t''}$ will start at $x_{t'}$ and end at $x_{t''}$ with probability 1.

For our particular example, we set $t'=1$, $t''=2$, $x_1 = e^{-1}$ and $x_2 = e^{-2}$.

We can construct this implied auxiliary bridge SDE from the original SDE as follows:

In [4]:
t_start = 1.; t_end = 2;

bridge_1_kwargs = {'t_start': t_start,
                 't_end': t_end,
                 'x_end': x_0*np.exp(-rho*t_end)
                }
dh_bridge_1 = DelyonHuBridge(ou_sde, **bridge_1_kwargs)

This constructed bridge is an instance of the class `AuxiliaryBridge` which is an instance of the class `SDE`. Therefore, it is possible to generate simulations and construct distribution objects for this auxiliary SDE.

In [5]:
dh_bridge_1_sim_kwargs = {'t_start': 0.0,
                        't_end': t_end - t_start,
                        'x_start': x_0*np.exp(-rho*t_start),
                        'num': 1000}

dh_bridge_1_sims = dh_bridge_1.simulate(size=10, **dh_bridge_1_sim_kwargs)

TypeError: AuxiliaryBridge.simulate() got an unexpected keyword argument 't_start'

In [ ]:
plot_simulations(dh_bridge_1_sims);

The `simulate` method of an auxiliary bridge SDE is modified, so that if the simulation runs to the end point, the end point that would have been generated by the (approximate) simulation is replaced with the end point that the function should hit with probability 1. In practice, the simulations, unless modified, will not necessarily hit this end point, due to the introduction of an approximate numerical scheme.

Note: If when implementing the `simulate` method, the `t_end` parameter is set to be greater than $\Delta t = t'' - t'$, then an `Exception` will be raised, as the drift and diffusion co-efficients of the auxiliary SDE are not defined for $t>\Delta t$.

We now demonstrate the implementation of the $F_{t', t''}^{(DH)}(x_{t'}, x_{t''},\quad  \cdot  \quad )$ function in this particular case, using the `transform_X_to_W` method on the auxiliary SDE:

In [ ]:
transform_kwargs = {'t_start': 0.,
                    'x_start': x_0*np.exp(-rho*t_start),
                    'transform_end_point': True
                   }

W = dh_bridge_1.transform_X_to_W(dh_bridge_1_sims, **transform_kwargs)

Only the start point `x_start`, corresponding to $x_{t'}$ and the starting time `t_start`, corresponding to $t'$ need to be given, as the end point $x_{t''}$ is given when constructing the auxiliary bridge SDE, and the time interval $\Delta t = t'' - t'$ is implied by the `dtype` of in the input simulations, thus implying the input $t''$. 

In [ ]:
plot_simulations(W);

The transformed paths resemble simulations from a Brownian motion, which would imply that the function works as intended.

We also test the inverse transform, using the `transform_W_to_X` method:

In [ ]:
transform_kwargs = {'t_start': 0.,
                    'x_start': x_0*np.exp(-rho*t_start),
                    'transform_end_point': True
                   }

X_1 = dh_bridge_1.transform_W_to_X(W, **transform_kwargs)

In [ ]:
assert check_struct_array_equal(dh_bridge_1_sims, X_1)

# Test 2 - Auxiliary Bridge transforms for vectorised triples $(x_{i-1}, x_{i}, x_{[t_{i-1}, t_i]})$

One of the uses of the function $F_{t', t''}^{(DH)}(x_{t'}, x_{t''},\quad  \cdot  \quad )$ in an SMC algorithm, is in the simulation of particles from the kernel $P_i[dx'_i, x'_{i-1}] = P_{i, 2}[dz_{[t_{i-1}, t_i]} , x_{i-1:i}]P_{i, 1}[dx_i, x_{i-1}]$.

In practice, we would have a vector of $N$ particles $x_{i-1}$, which represent the end points of the previous simulations of the path, after importance re-weighting has been applied. We would then generate a simulation of the SDE from $t_{i-1}$ to $t_i$ using an approximate numerical scheme, from the vector of different starting points. Thus, we would obtain $N$ triples of $(x_{i-1}, x_{i}, x_{[t_{i-1}, x_{t_i}]})$. Each of the pairs $(x_{i-1}, x_{i})$ represent an approximate sample from $P_{i, 1}[dx_i, x_{i-1}]$. For each of these $N$ particles, we apply the transform $F_{t_{i-1}, t_{i}}^{(DH)}(x_{i-1}, x_{i},\quad  \cdot  \quad )$ to the path $x_{[t_{i-1}, x_{t_i}]}$ to obtain the approximate sample from the kernel $P_{i, 2}[dz_{[t_{i-1}, t_i]} , x_{i-1:i}]$. Thus, an implementation of $F_{t_{i-1}, t_{i}}^{(DH)}(x_{i-1}, x_{i},\quad  \cdot  \quad )$ that is vectorised across different values of the triple $(x_{i-1}, x_{i}, x_{[t_{i-1}, x_{t_i}]})$ will be required to simulate from the kernel.  

We now test this vectorised implementation below. We test it by implementing it as it would be used to simulate from the kernel $P_i[dx'_i, x'_{i-1}]$. We again work with the univariate Ornstein-Uhlenbeck process defined by the following SDE:

$$dX_t = \rho(\mu - X_t)dt + \phi dW_t$$


We test a vectorised implementation for the implied auxiliary bridge transforms from $t_{i-1} = 1$ to $t_{i} = 2$. To generate appropriate starting points $x_{i-1}^{(j)}$, we generate simulations from the marginal distribution of SDE solution at $t_{i-1}$. This can be derived analytically, and is given by:

$$X_t \sim \mathcal{N}(x_0 e^{-\rho t} + \mu (1 - e^{-\rho t}), \frac{\phi^2}{2\rho} (1 - e^{-2\rho t}))$$

In [ ]:
N = 10

x_start_mean = x_0 * np.exp(-rho*t_start) + mu*(1-np.exp(-rho*t_start))
x_start_var = 0.5 * (phi ** 2) * (1 - np.exp(-2*rho*t_start)) * (rho ** -1)

x_starts = np.random.normal(loc=x_start_mean, scale=np.sqrt(x_start_var), size=N)

We simulate $N$ sample paths from the OU SDE, from $t_{i-1} = 1$ to $t_{i} = 2$:

In [ ]:
ou_sim_kwargs = {'t_start': t_start,
                 't_end': t_end,
                 'x_start': x_starts,
                 'num': 1000
                 }

ou_sims = ou_sde.simulate(size=N, **ou_sim_kwargs)

In [ ]:
plot_simulations(ou_sims, t_start = t_start);

These simulations are the vectorised triples $(x_{i-1}, x_{i}, x_{[t_{i-1}, t_i]})$ to which we need to apply a vectorised implentation of transforms $F_{t', t''}^{(DH)}(x_{t'}, x_{t''},\quad  \cdot  \quad )$ to obtain approximate samples from $P_i[dx'_i, x'_{i-1}]$. We construct an auxiliary bridge SDE object that has this vectorised implementation as a method as below:

In [ ]:
ou_sims_end_pts = ou_sims['1.0']

bridge_2_kwargs = {'t_start': t_start,
                   't_end': t_end,
                   'x_end': ou_sims_end_pts
                   }

dh_bridge_2 = DelyonHuBridge(ou_sde, **bridge_2_kwargs)

The constructed object `dh_bridge_2` of class `AuxiliaryBridge`, represents $N$ different auxiliary bridge SDEs. The point that the auxiliary bridge SDE hits with probability one is a part of the auxiliary SDE drift coefficient.

We can generate simulations from each of these auxiliary SDEs by calling the `simulate` method as exemplified below:

In [ ]:
dh_bridge_2_sim_kwargs = {'t_start': 0.0,
                        't_end': t_end - t_start,
                        'x_start': x_starts,
                        'num': 1000}

dh_bridge_2_sims = dh_bridge_2.simulate(size=N, **dh_bridge_2_sim_kwargs)

The `xstarts` key word argument input is in this case a vector, which corresponds to starting points of the OU SDE simulations $x_{i-1}^{(j)}$.

In [ ]:
plot_simulations(dh_bridge_2_sims);

The vectorised implementation of the transforms $F_{t', t''}^{(DH)}(x_{t'}, x_{t''},\quad  \cdot  \quad )$ for each of the $N$ auxiliary SDEs can be implemented via the `transform_X_to_W` method:

In [ ]:
transform_kwargs = {'t_start': 0.,
                    'x_start': x_starts,
                    'transform_end_point': False
                   }

Z = dh_bridge_2.transform_X_to_W(ou_sims, **transform_kwargs)
W = dh_bridge_2.transform_X_to_W(dh_bridge_2_sims, **transform_kwargs)

These transforms, when applied to the simulations from the $N$ auxiliary bridges, will output simulations from the standard Brownian motion. When applied to the path component of the triples $(x_{i-1}, x_{i}, x_{[t_{i-1}, t_i]})$, we will recover approximate simulations from the kernel $P_{i, 2}[dz_{[t_{i-1}, t_i]} , x_{i-1:i}]$

Note that the `transform_end_point` key word argument is set to `False` - this is important for the purpose of generating simulations from the kernel $P_i[dx'_i, x'_{i-1}]$, as the end points $x_i$ need to be retained.

We plot the simulations from the path component of the simulations from $P_i[dx'_i, x'_{i-1}]$ below:

In [ ]:
plot_simulations(Z, end_point=False);

Each of these paths have been simulated from different distributions, but the distributions of each of these paths are absolutely integrable with respect to the Weiner measure.

We additionally plot the simulations from the transforms applied to the simulations from the $N$ auxiliary bridges below:

In [ ]:
plot_simulations(W, end_point=False);

These simulated paths should all have the same distribution, which is the Weiner measure. The plotted simulations above reflect this.

Finally, we test the inverse of the vectorised transform, which can be implemented with the method `transform_W_to_X`:

In [ ]:
transform_kwargs = {'t_start': 0.,
                    'x_start': x_starts,
                    'transform_end_point': False
                   }

X_2 = dh_bridge_2.transform_W_to_X(W, **transform_kwargs)
X_tilde = dh_bridge_2.transform_W_to_X(Z, **transform_kwargs)

We then verify that both in the case of the simulations from the OU SDE and in the case of the simulations from the auxiliary bridge processes, that applying the vectorised transform, followed by its inverse, returns the original paths:

In [ ]:
assert check_struct_array_equal(dh_bridge_2_sims, X_2)
assert check_struct_array_equal(ou_sims, X_tilde)